In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
driver = webdriver.Chrome()

In [10]:
# 웹 페이지 열기
driver.get('https://www.ypbooks.co.kr/book.yp?bookcd=101253548') # 1.6초

In [4]:
# driver.get('https://product.kyobobook.co.kr/detail/S000211352514') # 교보문고는 5.2초, 교보문고가 오래걸림

In [11]:
    # 버튼이 로드될 때까지 대기하고 클릭
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "call_pop_stock"))
).click()

In [23]:
# 'store' 클래스를 가진 모든 td 태그 찾기
store_elements = driver.find_elements(By.CSS_SELECTOR, "tbody .store")



In [24]:
len(store_elements)

43

In [25]:
element=store_elements[0]

In [26]:
element

<selenium.webdriver.remote.webelement.WebElement (session="5ea9169ba50437f4179dfb50e37d0e6e", element="48FE20FD8C893B22E5C1189181A43B3F_element_1275")>

In [29]:
store_name = element.find_element(By.TAG_NAME, "strong").text

In [30]:
store_name

'가산마리오'

In [31]:
stock = element.find_element(By.CLASS_NAME, "stock").text

In [32]:
stock

'62'

In [41]:
driver.quit()

In [42]:
driver = webdriver.Chrome()
driver.get('https://www.ypbooks.co.kr/book.yp?bookcd=101253548')
WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "call_pop_stock"))
        ).click()
# 동적 컨텐츠 로드 대기
WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody .store"))
)

store_elements = driver.find_elements(By.CSS_SELECTOR, "tbody .store")

In [44]:
stores_stock = {}

In [45]:
for element in store_elements:
    store_name = element.find_element(By.TAG_NAME, "strong").text
    stock = element.find_element(By.CLASS_NAME, "stock").text

    stores_stock[store_name] = stock

In [47]:
driver.quit()

# 함수화

In [48]:
def yp_stock(url):
    driver = webdriver.Chrome()
    driver.get(url)
    stores_stock = {}
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "call_pop_stock"))
        ).click()
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody .store"))
        )
        store_elements = driver.find_elements(By.CSS_SELECTOR, "tbody .store")

        for element in store_elements:
            store_name = element.find_element(By.TAG_NAME, "strong").text
            stock = element.find_element(By.CLASS_NAME, "stock").text

            stores_stock[store_name] = stock
    finally:
        driver.quit()

    return stores_stock

In [49]:
check = yp_stock('https://www.ypbooks.co.kr/book.yp?bookcd=101253548') # 3.5초 걸림

In [50]:
check

{'가산마리오': '62',
 '강남역': '56',
 '강남포스코': '49',
 '경산이마트': '27',
 '광복롯데': '47',
 '광주터미널': '211',
 '구리롯데아울렛': '24',
 '구미롯데마트': '33',
 '군산롯데아울렛': '33',
 '김포공항롯데': '118',
 '대구역롯데': '58',
 '대전터미널': '42',
 '동탄롯데': '57',
 '마산롯데': '21',
 '목포터미널': '20',
 '미아롯데': '30',
 '부산남포': '42',
 '부산대': '24',
 '부산하단': '18',
 '분당서현': '76',
 '사당역': '112',
 '세종이마트': '21',
 '센텀시티': '55',
 '수원NC': '30',
 '스타필드고양': '105',
 '스타필드시티위례': '60',
 '스타필드안성': '26',
 '스타필드코엑스몰': '224',
 '스타필드하남': '158',
 '신림타임스트림': '46',
 '여의도IFC몰': '253',
 '왕십리역': '83',
 '용산아이파크몰': '46',
 '유성': '60',
 '의정부신세계': '96',
 '인천스퀘어원': '56',
 '인천터미널': '59',
 '전주터미널': '20',
 '종각종로': '105',
 '죽전이마트': '15',
 '천안': '33',
 '청주': '54',
 '홍대': '69'}

# 추가 작업 (네트워크 뒷단에 돌고 있는 자바스크립트 코드 뜯기)

In [51]:
from requests.sessions import Session

In [52]:
url = 'https://www.ypbooks.co.kr/ypbooks_mobile/sub/mBranchStockLoc.jsp?bookCd=101253548&bookCost=17000&josgaNo1=30'
session = Session()
resp = session.request('GET', url)

In [54]:
resp.text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t<script src="/ypbooks_mobile/js/jquery-1.12.4.min.js"></script>\r\n</head>\r\n<body>\r\n\r\n\t<table class="table" summary="">\r\n\t\t<p>영업점 재고·위치 (지점명 가나다순)</p>\r\n\t\t<p class="notice">영업점 서가 위치를 확인하려면 해당 <strong class="underline">영업점을 클릭</strong>하세요.<br>\r\n\t\t실시간 재고수량이 변하므로 <strong class="underline">전화확인 후 방문</strong>바랍니다.</p>\r\n\t\t<colgroup>\r\n\t\t\t<col style="width:16.65%">\r\n\t\t\t<col style="width:16.65%">\r\n\t\t\t<col style="width:16.65%">\r\n\t\t\t<col style="width:16.65%">\r\n\t\t\t<col style="width:16.65%">\r\n\t\t\t<col style="width:16.65%">\r\n\t\t</colgroup>\r\n\t\t<tbody>\r\n<tr>\r\n\t\t\t\t<td class="store"><a href="javascript:fLocationInfo(4900);"><strong>가산마리오</strong><br><span class="stock">62</span></a></td>\r\n\t\t\t\t<td class="store"><a href="javascript:fLocationInfo(3700);"><strong>강남역</strong><br><span class="stock">56</span></a></td>\r\n\t\t\t\t<

In [55]:
import requests
from bs4 import BeautifulSoup

# URL 설정
url = 'https://www.ypbooks.co.kr/ypbooks_mobile/sub/mBranchStockLoc.jsp?bookCd=101253548&bookCost=17000&josgaNo1=30'

# requests를 사용하여 웹 페이지 가져오기
resp = requests.get(url)

# BeautifulSoup으로 HTML 파싱
soup = BeautifulSoup(resp.text, 'html.parser')

# 결과를 저장할 딕셔너리
stores_stock = {}

# 'store' 클래스를 가진 모든 td 태그 찾기
store_elements = soup.find_all('td', class_='store')

for element in store_elements:
    store_name = element.find('strong').text
    stock = element.find('span', class_='stock').text
    stores_stock[store_name] = stock



In [57]:
stores_stock # 확실히 개빠름 0.4초만에

{'가산마리오': '62',
 '강남역': '56',
 '강남포스코': '49',
 '경산이마트': '27',
 '광복롯데': '47',
 '광주터미널': '211',
 '구리롯데아울렛': '24',
 '구미롯데마트': '33',
 '군산롯데아울렛': '33',
 '김포공항롯데': '118',
 '대구역롯데': '58',
 '대전터미널': '42',
 '동탄롯데': '57',
 '마산롯데': '21',
 '목포터미널': '20',
 '미아롯데': '30',
 '부산남포': '42',
 '부산대': '24',
 '부산하단': '18',
 '분당서현': '76',
 '사당역': '112',
 '세종이마트': '21',
 '센텀시티': '55',
 '수원NC': '30',
 '스타필드고양': '105',
 '스타필드시티위례': '60',
 '스타필드안성': '26',
 '스타필드코엑스몰': '224',
 '스타필드하남': '158',
 '신림타임스트림': '46',
 '여의도IFC몰': '253',
 '왕십리역': '83',
 '용산아이파크몰': '46',
 '유성': '60',
 '의정부신세계': '96',
 '인천스퀘어원': '56',
 '인천터미널': '59',
 '전주터미널': '20',
 '종각종로': '105',
 '죽전이마트': '15',
 '천안': '33',
 '청주': '54',
 '홍대': '69'}

In [ ]:
&josgaNo1=36

In [64]:
url = 'https://www.ypbooks.co.kr/ypbooks_mobile/sub/mBranchStockLoc.jsp?bookCd=101250576'

resp = requests.get(url)

soup = BeautifulSoup(resp.text, 'html.parser')

In [65]:
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<script src="/ypbooks_mobile/js/jquery-1.12.4.min.js"></script>
</head>
<body>
<table class="table" summary="">
<p>영업점 재고·위치 (지점명 가나다순)</p>
<p class="notice">영업점 서가 위치를 확인하려면 해당 <strong class="underline">영업점을 클릭</strong>하세요.<br/>
		실시간 재고수량이 변하므로 <strong class="underline">전화확인 후 방문</strong>바랍니다.</p>
<colgroup>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
</colgroup>
<tbody>
</tbody>
</table>
<button class="close-table" onclick="javascript:swtbranchview(2)" type="button">닫기<i></i></button>
</body>
</html>

In [66]:
url = 'https://www.ypbooks.co.kr/ypbooks_mobile/sub/mBranchStockLoc.jsp?bookCd=101250576&bookCost=30000'

resp = requests.get(url)

soup = BeautifulSoup(resp.text, 'html.parser')

In [67]:
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<script src="/ypbooks_mobile/js/jquery-1.12.4.min.js"></script>
</head>
<body>
<table class="table" summary="">
<p>영업점 재고·위치 (지점명 가나다순)</p>
<p class="notice">영업점 서가 위치를 확인하려면 해당 <strong class="underline">영업점을 클릭</strong>하세요.<br/>
		실시간 재고수량이 변하므로 <strong class="underline">전화확인 후 방문</strong>바랍니다.</p>
<colgroup>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
<col style="width:16.65%"/>
</colgroup>
<tbody>
<tr>
<td class="store"><a href="javascript:fLocationInfo(4900);"><strong>가산마리오</strong><br/><span class="stock">1</span></a></td>
<td class="store"><a href="javascript:fLocationInfo(3700);"><strong>강남역</strong><br/><span class="stock">2</span></a></td>
<td class="store"><a href="javascript:fLocationInfo(8260);"><strong>강남포스코</strong><br/><span class="stock">1</span></a></td></tr><tr>
<td class="store"><strong>경산이마트</strong><br/><span class="stock">0</spa

In [ ]:
# 번호, 가격만 가져오면 된다.

# 함수화

In [72]:
code = 101250576
price = 30000

In [69]:
def yp_stock(code, price):
    url = f'https://www.ypbooks.co.kr/ypbooks_mobile/sub/mBranchStockLoc.jsp?bookCd={code}&bookCost={price}'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    stores_stock = {}
    store_elements = soup.find_all('td', class_='store')

    for element in store_elements:
        store_name = element.find('strong').text
        stock = element.find('span', class_='stock').text
        stores_stock[store_name] = stock
    
    return stores_stock


In [73]:
check = yp_stock(code, price)

In [74]:
check # 0.2초 걸림

{'가산마리오': '1',
 '강남역': '2',
 '강남포스코': '1',
 '경산이마트': '0',
 '광복롯데': '0',
 '광주터미널': '1',
 '구리롯데아울렛': '0',
 '구미롯데마트': '1',
 '군산롯데아울렛': '1',
 '김포공항롯데': '1',
 '대구역롯데': '0',
 '대전터미널': '2',
 '동탄롯데': '1',
 '마산롯데': '0',
 '목포터미널': '0',
 '미아롯데': '0',
 '부산남포': '0',
 '부산대': '1',
 '부산하단': '1',
 '분당서현': '4',
 '사당역': '1',
 '세종이마트': '0',
 '센텀시티': '0',
 '수원NC': '0',
 '스타필드고양': '0',
 '스타필드시티위례': '0',
 '스타필드안성': '0',
 '스타필드코엑스몰': '1',
 '스타필드하남': '0',
 '신림타임스트림': '0',
 '여의도IFC몰': '1',
 '왕십리역': '1',
 '용산아이파크몰': '2',
 '유성': '1',
 '의정부신세계': '3',
 '인천스퀘어원': '0',
 '인천터미널': '0',
 '전주터미널': '1',
 '종각종로': '1',
 '죽전이마트': '0',
 '천안': '0',
 '청주': '2',
 '홍대': '2'}